In [0]:
dbutils.fs.mkdirs("dbfs:/FileStore/source")
dbutils.fs.mkdirs("dbfs:/FileStore/target")

Out[2]: True

In [0]:
source_df = spark.read.option("inferSchema", "true").csv("dbfs:/FileStore/source/growth_morhpho_scd-1.csv", header=True)

In [0]:
display(source_df)

GenotypeName,StemAnthocyanin,AnthocyaninIntensity,StemPubescence,StemPubescenceIntensity,LeafBladeColour,LeafBladeColourIntensity,ColourOfVein,IntensityOfColour8,SpineonLeaf,FlowerColour,FruitingPattern,FruitColour,IntensityofColour13,Stripes,StripesDensity,FruitPatches,SpinesInCalyx,DensityOfSpininess,FruitShape
101,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Purple,Cluster,Purple,Medium,Present,Sparse,Present,Absent,Absent,Globular
102,Present,Medium,Present,Medium,Green,Medium,Purple,Medium,Absent,Purple,Solitary,Purple,Dark,Present,Sparse,Absent,Absent,Absent,Globular
104,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Greenish_White,Solitary,Green,Med ium,Absent,Absent,Absent,Present,Absent,Club
105,Present,Weak,Present,Medium,Green,Medium,Purple,Light,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate
106,Present,Medium,Present,Weak,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Absent,Absent,Absent,Present,Medium,Club
108,Present,Strong,Present,Medium,Green,Light,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Present,Present,Medium,Club
109,Present,Medium,Present,Medium,Green,Dark,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate
110,Absent,Absent,Present,Strong,Green,Medium,Green,Medium,Absent,Purple,Solitary,Green,Medium,Absent,Absent,Present,Absent,Absent,Globular
111,Present,Weak,Present,Medium,Green,Medium,Green,Medium,Absent,Purple,Solitary,Purple,Medium,Present,Strong,Absent,Present,Medium,Ovoid
112,Absent,Absent,Present,Medium,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Present,Sparse,Present,Absent,Absent,Obovate


In [0]:
DATE_FORMAT = "yyyy-MM-dd"
future_date = "9999-12-31"
source_url = "dbfs:/FileStore/source/growth_morhpho_scd.csv"
destination_url = "dbfs:/FileStore/target/"
primary_key = ["GenotypeName"]
slowly_changing_cols = [ "FruitShape","StemAnthocyanin","StemPubescence"]
implementation_cols = ["effective_date","end_date","active_flag"]

In [0]:
from pyspark.sql.window import Window

In [0]:
window_spec = Window.orderBy("GenotypeName")

In [0]:
from pyspark.sql.functions import *
enhanced_growth_df = source_df \
.withColumn("surrogate_key",row_number().over(window_spec)) \
.withColumn("effective_date",date_format(current_date(), DATE_FORMAT)) \
.withColumn("end_date",date_format(lit(future_date), DATE_FORMAT)) \
.withColumn("active_flag", lit(True))

In [0]:
display(enhanced_growth_df)

GenotypeName,StemAnthocyanin,AnthocyaninIntensity,StemPubescence,StemPubescenceIntensity,LeafBladeColour,LeafBladeColourIntensity,ColourOfVein,IntensityOfColour8,SpineonLeaf,FlowerColour,FruitingPattern,FruitColour,IntensityofColour13,Stripes,StripesDensity,FruitPatches,SpinesInCalyx,DensityOfSpininess,FruitShape,surrogate_key,effective_date,end_date,active_flag
101,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Purple,Cluster,Purple,Medium,Present,Sparse,Present,Absent,Absent,Obovate,1,2024-08-13,9999-12-31,true
102,Present,Medium,Present,Medium,Green,Medium,Purple,Medium,Absent,Purple,Solitary,Purple,Dark,Present,Sparse,Absent,Absent,Absent,Globular,2,2024-08-13,9999-12-31,true
103,Absent,Absent,Present,Medium,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Present,Sparse,Present,Absent,Absent,Obovate,3,2024-08-13,9999-12-31,true
104,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Greenish_White,Solitary,Green,Med ium,Absent,Absent,Absent,Present,Absent,Club,4,2024-08-13,9999-12-31,true
105,Present,Weak,Present,Medium,Green,Medium,Purple,Light,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate,5,2024-08-13,9999-12-31,true
106,Present,Medium,Present,Weak,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Absent,Absent,Absent,Present,Medium,Club,6,2024-08-13,9999-12-31,true
107,Present,Weak,Present,Medium,Green,Medium,Green,Medium,Absent,Purple,Solitary,Purple,Medium,Present,Strong,Absent,Present,Medium,Ovoid,7,2024-08-13,9999-12-31,true
108,Present,Strong,Present,Medium,Green,Light,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Present,Present,Medium,Ovoid,8,2024-08-13,9999-12-31,true
109,Present,Medium,Present,Medium,Green,Dark,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate,9,2024-08-13,9999-12-31,true
110,Absent,Absent,Present,Strong,Green,Medium,Green,Medium,Absent,Purple,Solitary,Green,Medium,Absent,Absent,Present,Absent,Absent,Globular,10,2024-08-13,9999-12-31,true


In [0]:
enhanced_growth_df.repartition(1).write.mode("overwrite").format("csv").option("header", "true").save("dbfs:/FileStore/target/")

In [0]:
target_df = spark.read.option("inferSchema", "true").csv("dbfs:/FileStore/target/", header= True)
display(target_df)

GenotypeName,StemAnthocyanin,AnthocyaninIntensity,StemPubescence,StemPubescenceIntensity,LeafBladeColour,LeafBladeColourIntensity,ColourOfVein,IntensityOfColour8,SpineonLeaf,FlowerColour,FruitingPattern,FruitColour,IntensityofColour13,Stripes,StripesDensity,FruitPatches,SpinesInCalyx,DensityOfSpininess,FruitShape,surrogate_key,effective_date,end_date,active_flag
101,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Purple,Cluster,Purple,Medium,Present,Sparse,Present,Absent,Absent,Obovate,1,2024-08-13,9999-12-31,true
102,Present,Medium,Present,Medium,Green,Medium,Purple,Medium,Absent,Purple,Solitary,Purple,Dark,Present,Sparse,Absent,Absent,Absent,Globular,2,2024-08-13,9999-12-31,true
103,Absent,Absent,Present,Medium,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Present,Sparse,Present,Absent,Absent,Obovate,3,2024-08-13,9999-12-31,true
104,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Greenish_White,Solitary,Green,Med ium,Absent,Absent,Absent,Present,Absent,Club,4,2024-08-13,9999-12-31,true
105,Present,Weak,Present,Medium,Green,Medium,Purple,Light,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate,5,2024-08-13,9999-12-31,true
106,Present,Medium,Present,Weak,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Absent,Absent,Absent,Present,Medium,Club,6,2024-08-13,9999-12-31,true
107,Present,Weak,Present,Medium,Green,Medium,Green,Medium,Absent,Purple,Solitary,Purple,Medium,Present,Strong,Absent,Present,Medium,Ovoid,7,2024-08-13,9999-12-31,true
108,Present,Strong,Present,Medium,Green,Light,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Present,Present,Medium,Ovoid,8,2024-08-13,9999-12-31,true
109,Present,Medium,Present,Medium,Green,Dark,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate,9,2024-08-13,9999-12-31,true
110,Absent,Absent,Present,Strong,Green,Medium,Green,Medium,Absent,Purple,Solitary,Green,Medium,Absent,Absent,Present,Absent,Absent,Globular,10,2024-08-13,9999-12-31,true


In [0]:
display(source_df)

GenotypeName,StemAnthocyanin,AnthocyaninIntensity,StemPubescence,StemPubescenceIntensity,LeafBladeColour,LeafBladeColourIntensity,ColourOfVein,IntensityOfColour8,SpineonLeaf,FlowerColour,FruitingPattern,FruitColour,IntensityofColour13,Stripes,StripesDensity,FruitPatches,SpinesInCalyx,DensityOfSpininess,FruitShape
101,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Purple,Cluster,Purple,Medium,Present,Sparse,Present,Absent,Absent,Globular
102,Present,Medium,Present,Medium,Green,Medium,Purple,Medium,Absent,Purple,Solitary,Purple,Dark,Present,Sparse,Absent,Absent,Absent,Globular
104,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Greenish_White,Solitary,Green,Med ium,Absent,Absent,Absent,Present,Absent,Club
105,Present,Weak,Present,Medium,Green,Medium,Purple,Light,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate
106,Present,Medium,Present,Weak,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Absent,Absent,Absent,Present,Medium,Club
108,Present,Strong,Present,Medium,Green,Light,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Present,Present,Medium,Club
109,Present,Medium,Present,Medium,Green,Dark,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate
110,Absent,Absent,Present,Strong,Green,Medium,Green,Medium,Absent,Purple,Solitary,Green,Medium,Absent,Absent,Present,Absent,Absent,Globular
111,Present,Weak,Present,Medium,Green,Medium,Green,Medium,Absent,Purple,Solitary,Purple,Medium,Present,Strong,Absent,Present,Medium,Ovoid
112,Absent,Absent,Present,Medium,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Present,Sparse,Present,Absent,Absent,Obovate


In [0]:
active_target_df = target_df.where(col("active_flag")== True)

In [0]:
inactive_target_df = target_df.where(col("active_flag")== False)

In [0]:
display(active_target_df)

GenotypeName,StemAnthocyanin,AnthocyaninIntensity,StemPubescence,StemPubescenceIntensity,LeafBladeColour,LeafBladeColourIntensity,ColourOfVein,IntensityOfColour8,SpineonLeaf,FlowerColour,FruitingPattern,FruitColour,IntensityofColour13,Stripes,StripesDensity,FruitPatches,SpinesInCalyx,DensityOfSpininess,FruitShape,surrogate_key,effective_date,end_date,active_flag
101,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Purple,Cluster,Purple,Medium,Present,Sparse,Present,Absent,Absent,Obovate,1,2024-08-13,9999-12-31,true
102,Present,Medium,Present,Medium,Green,Medium,Purple,Medium,Absent,Purple,Solitary,Purple,Dark,Present,Sparse,Absent,Absent,Absent,Globular,2,2024-08-13,9999-12-31,true
103,Absent,Absent,Present,Medium,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Present,Sparse,Present,Absent,Absent,Obovate,3,2024-08-13,9999-12-31,true
104,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Greenish_White,Solitary,Green,Med ium,Absent,Absent,Absent,Present,Absent,Club,4,2024-08-13,9999-12-31,true
105,Present,Weak,Present,Medium,Green,Medium,Purple,Light,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate,5,2024-08-13,9999-12-31,true
106,Present,Medium,Present,Weak,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Absent,Absent,Absent,Present,Medium,Club,6,2024-08-13,9999-12-31,true
107,Present,Weak,Present,Medium,Green,Medium,Green,Medium,Absent,Purple,Solitary,Purple,Medium,Present,Strong,Absent,Present,Medium,Ovoid,7,2024-08-13,9999-12-31,true
108,Present,Strong,Present,Medium,Green,Light,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Present,Present,Medium,Ovoid,8,2024-08-13,9999-12-31,true
109,Present,Medium,Present,Medium,Green,Dark,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate,9,2024-08-13,9999-12-31,true
110,Absent,Absent,Present,Strong,Green,Medium,Green,Medium,Absent,Purple,Solitary,Green,Medium,Absent,Absent,Present,Absent,Absent,Globular,10,2024-08-13,9999-12-31,true


In [0]:
display(active_target_df.join(source_df, 'GenotypeName', 'full_outer'))

GenotypeName,StemAnthocyanin,AnthocyaninIntensity,StemPubescence,StemPubescenceIntensity,LeafBladeColour,LeafBladeColourIntensity,ColourOfVein,IntensityOfColour8,SpineonLeaf,FlowerColour,FruitingPattern,FruitColour,IntensityofColour13,Stripes,StripesDensity,FruitPatches,SpinesInCalyx,DensityOfSpininess,FruitShape,surrogate_key,effective_date,end_date,active_flag,StemAnthocyanin,AnthocyaninIntensity,StemPubescence,StemPubescenceIntensity,LeafBladeColour,LeafBladeColourIntensity,ColourOfVein,IntensityOfColour8,SpineonLeaf,FlowerColour,FruitingPattern,FruitColour,IntensityofColour13,Stripes,StripesDensity,FruitPatches,SpinesInCalyx,DensityOfSpininess,FruitShape
101,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Purple,Cluster,Purple,Medium,Present,Sparse,Present,Absent,Absent,Obovate,1,2024-08-13,9999-12-31,true,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Purple,Cluster,Purple,Medium,Present,Sparse,Present,Absent,Absent,Globular
102,Present,Medium,Present,Medium,Green,Medium,Purple,Medium,Absent,Purple,Solitary,Purple,Dark,Present,Sparse,Absent,Absent,Absent,Globular,2,2024-08-13,9999-12-31,true,Present,Medium,Present,Medium,Green,Medium,Purple,Medium,Absent,Purple,Solitary,Purple,Dark,Present,Sparse,Absent,Absent,Absent,Globular
103,Absent,Absent,Present,Medium,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Present,Sparse,Present,Absent,Absent,Obovate,3,2024-08-13,9999-12-31,true,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
104,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Greenish_White,Solitary,Green,Med ium,Absent,Absent,Absent,Present,Absent,Club,4,2024-08-13,9999-12-31,true,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Greenish_White,Solitary,Green,Med ium,Absent,Absent,Absent,Present,Absent,Club
105,Present,Weak,Present,Medium,Green,Medium,Purple,Light,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate,5,2024-08-13,9999-12-31,true,Present,Weak,Present,Medium,Green,Medium,Purple,Light,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate
106,Present,Medium,Present,Weak,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Absent,Absent,Absent,Present,Medium,Club,6,2024-08-13,9999-12-31,true,Present,Medium,Present,Weak,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Absent,Absent,Absent,Present,Medium,Club
107,Present,Weak,Present,Medium,Green,Medium,Green,Medium,Absent,Purple,Solitary,Purple,Medium,Present,Strong,Absent,Present,Medium,Ovoid,7,2024-08-13,9999-12-31,true,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
108,Present,Strong,Present,Medium,Green,Light,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Present,Present,Medium,Ovoid,8,2024-08-13,9999-12-31,true,Present,Strong,Present,Medium,Green,Light,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Present,Present,Medium,Club
109,Present,Medium,Present,Medium,Green,Dark,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate,9,2024-08-13,9999-12-31,true,Present,Medium,Present,Medium,Green,Dark,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate
110,Absent,Absent,Present,Strong,Green,Medium,Green,Medium,Absent,Purple,Solitary,Green,Medium,Absent,Absent,Present,Absent,Absent,Globular,10,2024-08-13,9999-12-31,true,Absent,Absent,Present,Strong,Green,Medium,Green,Medium,Absent,Purple,Solitary,Green,Medium,Absent,Absent,Present,Absent,Absent,Globular


In [0]:
def column_renamer(df, suffix, append):
   
    if append:
        new_column_names = list(map(lambda x: x+suffix, df.columns))
        
    else:
        new_column_names = list(map(lambda x: x.replace(suffix,""), df.columns))
        
    return df.toDF(*new_column_names)

def get_hash(df, keys_list):
 
    columns = [col(column) for column in keys_list]
    
    if columns:
        return df.withColumn("hash_md5", md5(concat_ws("", *columns)))
    else:
        return df.withColumn("hash_md5", md5(lit(1)))

In [0]:
active_target_df_hash = column_renamer(get_hash(active_target_df, slowly_changing_cols), suffix="_target", append=True)
source_df_hash = column_renamer(get_hash(source_df, slowly_changing_cols), suffix="_source", append=True)


In [0]:
active_target_rename_df = column_renamer(get_hash(active_target_df, slowly_changing_cols), suffix="_target", append=True)
source_df_rename_df  = column_renamer(get_hash(source_df, slowly_changing_cols), suffix="_source", append=True)
display(active_target_rename_df)
display(source_df_rename_df)

GenotypeName_target,StemAnthocyanin_target,AnthocyaninIntensity_target,StemPubescence_target,StemPubescenceIntensity_target,LeafBladeColour_target,LeafBladeColourIntensity_target,ColourOfVein_target,IntensityOfColour8_target,SpineonLeaf_target,FlowerColour_target,FruitingPattern_target,FruitColour_target,IntensityofColour13_target,Stripes_target,StripesDensity_target,FruitPatches_target,SpinesInCalyx_target,DensityOfSpininess_target,FruitShape_target,surrogate_key_target,effective_date_target,end_date_target,active_flag_target,hash_md5_target
101,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Purple,Cluster,Purple,Medium,Present,Sparse,Present,Absent,Absent,Obovate,1,2024-08-13,9999-12-31,true,d0b11a8e90cd207c0e1ba79f546668db
102,Present,Medium,Present,Medium,Green,Medium,Purple,Medium,Absent,Purple,Solitary,Purple,Dark,Present,Sparse,Absent,Absent,Absent,Globular,2,2024-08-13,9999-12-31,true,e15d870145693ffae88c2c44549cf920
103,Absent,Absent,Present,Medium,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Present,Sparse,Present,Absent,Absent,Obovate,3,2024-08-13,9999-12-31,true,d0b11a8e90cd207c0e1ba79f546668db
104,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Greenish_White,Solitary,Green,Med ium,Absent,Absent,Absent,Present,Absent,Club,4,2024-08-13,9999-12-31,true,58b417afa44eb2f7dc4f87715166adff
105,Present,Weak,Present,Medium,Green,Medium,Purple,Light,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate,5,2024-08-13,9999-12-31,true,a77aecffaaa9d9fe4a8bfa414846d51a
106,Present,Medium,Present,Weak,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Absent,Absent,Absent,Present,Medium,Club,6,2024-08-13,9999-12-31,true,d8021c4a568a96fdb891c62a5ffe9614
107,Present,Weak,Present,Medium,Green,Medium,Green,Medium,Absent,Purple,Solitary,Purple,Medium,Present,Strong,Absent,Present,Medium,Ovoid,7,2024-08-13,9999-12-31,true,8ced0877b07422df726d13d072463f22
108,Present,Strong,Present,Medium,Green,Light,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Present,Present,Medium,Ovoid,8,2024-08-13,9999-12-31,true,8ced0877b07422df726d13d072463f22
109,Present,Medium,Present,Medium,Green,Dark,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate,9,2024-08-13,9999-12-31,true,a77aecffaaa9d9fe4a8bfa414846d51a
110,Absent,Absent,Present,Strong,Green,Medium,Green,Medium,Absent,Purple,Solitary,Green,Medium,Absent,Absent,Present,Absent,Absent,Globular,10,2024-08-13,9999-12-31,true,de6de38ae45ec97976ca46c89ab29fa5


GenotypeName_source,StemAnthocyanin_source,AnthocyaninIntensity_source,StemPubescence_source,StemPubescenceIntensity_source,LeafBladeColour_source,LeafBladeColourIntensity_source,ColourOfVein_source,IntensityOfColour8_source,SpineonLeaf_source,FlowerColour_source,FruitingPattern_source,FruitColour_source,IntensityofColour13_source,Stripes_source,StripesDensity_source,FruitPatches_source,SpinesInCalyx_source,DensityOfSpininess_source,FruitShape_source,hash_md5_source
101,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Purple,Cluster,Purple,Medium,Present,Sparse,Present,Absent,Absent,Globular,de6de38ae45ec97976ca46c89ab29fa5
102,Present,Medium,Present,Medium,Green,Medium,Purple,Medium,Absent,Purple,Solitary,Purple,Dark,Present,Sparse,Absent,Absent,Absent,Globular,e15d870145693ffae88c2c44549cf920
104,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Greenish_White,Solitary,Green,Med ium,Absent,Absent,Absent,Present,Absent,Club,58b417afa44eb2f7dc4f87715166adff
105,Present,Weak,Present,Medium,Green,Medium,Purple,Light,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate,a77aecffaaa9d9fe4a8bfa414846d51a
106,Present,Medium,Present,Weak,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Absent,Absent,Absent,Present,Medium,Club,d8021c4a568a96fdb891c62a5ffe9614
108,Present,Strong,Present,Medium,Green,Light,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Present,Present,Medium,Club,d8021c4a568a96fdb891c62a5ffe9614
109,Present,Medium,Present,Medium,Green,Dark,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate,a77aecffaaa9d9fe4a8bfa414846d51a
110,Absent,Absent,Present,Strong,Green,Medium,Green,Medium,Absent,Purple,Solitary,Green,Medium,Absent,Absent,Present,Absent,Absent,Globular,de6de38ae45ec97976ca46c89ab29fa5
111,Present,Weak,Present,Medium,Green,Medium,Green,Medium,Absent,Purple,Solitary,Purple,Medium,Present,Strong,Absent,Present,Medium,Ovoid,8ced0877b07422df726d13d072463f22
112,Absent,Absent,Present,Medium,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Present,Sparse,Present,Absent,Absent,Obovate,d0b11a8e90cd207c0e1ba79f546668db


In [0]:
merged_df = active_target_rename_df.join(source_df_rename_df, col("GenotypeName_source") ==  col("GenotypeName_target") , "full_outer") \
.withColumn("Action", when(col("hash_md5_source") == col("hash_md5_target")  , 'NOCHANGE')\
.when(col("GenotypeName_source").isNull(), 'DELETE')\
.when(col("GenotypeName_target").isNull(), 'INSERT')\
.otherwise('UPDATE'))
display(merged_df)

GenotypeName_target,StemAnthocyanin_target,AnthocyaninIntensity_target,StemPubescence_target,StemPubescenceIntensity_target,LeafBladeColour_target,LeafBladeColourIntensity_target,ColourOfVein_target,IntensityOfColour8_target,SpineonLeaf_target,FlowerColour_target,FruitingPattern_target,FruitColour_target,IntensityofColour13_target,Stripes_target,StripesDensity_target,FruitPatches_target,SpinesInCalyx_target,DensityOfSpininess_target,FruitShape_target,surrogate_key_target,effective_date_target,end_date_target,active_flag_target,hash_md5_target,GenotypeName_source,StemAnthocyanin_source,AnthocyaninIntensity_source,StemPubescence_source,StemPubescenceIntensity_source,LeafBladeColour_source,LeafBladeColourIntensity_source,ColourOfVein_source,IntensityOfColour8_source,SpineonLeaf_source,FlowerColour_source,FruitingPattern_source,FruitColour_source,IntensityofColour13_source,Stripes_source,StripesDensity_source,FruitPatches_source,SpinesInCalyx_source,DensityOfSpininess_source,FruitShape_source,hash_md5_source,Action
101,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Purple,Cluster,Purple,Medium,Present,Sparse,Present,Absent,Absent,Obovate,1,2024-08-13,9999-12-31,true,d0b11a8e90cd207c0e1ba79f546668db,101,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Purple,Cluster,Purple,Medium,Present,Sparse,Present,Absent,Absent,Globular,de6de38ae45ec97976ca46c89ab29fa5,UPDATE
102,Present,Medium,Present,Medium,Green,Medium,Purple,Medium,Absent,Purple,Solitary,Purple,Dark,Present,Sparse,Absent,Absent,Absent,Globular,2,2024-08-13,9999-12-31,true,e15d870145693ffae88c2c44549cf920,102,Present,Medium,Present,Medium,Green,Medium,Purple,Medium,Absent,Purple,Solitary,Purple,Dark,Present,Sparse,Absent,Absent,Absent,Globular,e15d870145693ffae88c2c44549cf920,NOCHANGE
103,Absent,Absent,Present,Medium,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Present,Sparse,Present,Absent,Absent,Obovate,3,2024-08-13,9999-12-31,true,d0b11a8e90cd207c0e1ba79f546668db,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,DELETE
104,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Greenish_White,Solitary,Green,Med ium,Absent,Absent,Absent,Present,Absent,Club,4,2024-08-13,9999-12-31,true,58b417afa44eb2f7dc4f87715166adff,104,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Greenish_White,Solitary,Green,Med ium,Absent,Absent,Absent,Present,Absent,Club,58b417afa44eb2f7dc4f87715166adff,NOCHANGE
105,Present,Weak,Present,Medium,Green,Medium,Purple,Light,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate,5,2024-08-13,9999-12-31,true,a77aecffaaa9d9fe4a8bfa414846d51a,105,Present,Weak,Present,Medium,Green,Medium,Purple,Light,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate,a77aecffaaa9d9fe4a8bfa414846d51a,NOCHANGE
106,Present,Medium,Present,Weak,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Absent,Absent,Absent,Present,Medium,Club,6,2024-08-13,9999-12-31,true,d8021c4a568a96fdb891c62a5ffe9614,106,Present,Medium,Present,Weak,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Absent,Absent,Absent,Present,Medium,Club,d8021c4a568a96fdb891c62a5ffe9614,NOCHANGE
107,Present,Weak,Present,Medium,Green,Medium,Green,Medium,Absent,Purple,Solitary,Purple,Medium,Present,Strong,Absent,Present,Medium,Ovoid,7,2024-08-13,9999-12-31,true,8ced0877b07422df726d13d072463f22,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,DELETE
108,Present,Strong,Present,Medium,Green,Light,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Present,Present,Medium,Ovoid,8,2024-08-13,9999-12-31,true,8ced0877b07422df726d13d072463f22,108,Present,Strong,Present,Medium,Green,Light,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Present,Present,Medium,Club,d8021c4a568a96fdb891c62a5ffe9614,UPDATE
109,Present,Medium,Present

In [0]:
unchanged_records = column_renamer(merged_df.filter(col("action") == 'NOCHANGE'), suffix="_target", append=False)

In [0]:
print(unchanged_records.columns)

['GenotypeName', 'StemAnthocyanin', 'AnthocyaninIntensity', 'StemPubescence', 'StemPubescenceIntensity', 'LeafBladeColour', 'LeafBladeColourIntensity', 'ColourOfVein', 'IntensityOfColour8', 'SpineonLeaf', 'FlowerColour', 'FruitingPattern', 'FruitColour', 'IntensityofColour13', 'Stripes', 'StripesDensity', 'FruitPatches', 'SpinesInCalyx', 'DensityOfSpininess', 'FruitShape', 'surrogate_key', 'effective_date', 'end_date', 'active_flag', 'hash_md5', 'GenotypeName_source', 'StemAnthocyanin_source', 'AnthocyaninIntensity_source', 'StemPubescence_source', 'StemPubescenceIntensity_source', 'LeafBladeColour_source', 'LeafBladeColourIntensity_source', 'ColourOfVein_source', 'IntensityOfColour8_source', 'SpineonLeaf_source', 'FlowerColour_source', 'FruitingPattern_source', 'FruitColour_source', 'IntensityofColour13_source', 'Stripes_source', 'StripesDensity_source', 'FruitPatches_source', 'SpinesInCalyx_source', 'DensityOfSpininess_source', 'FruitShape_source', 'hash_md5_source', 'Action']


In [0]:
final_unchanged_data_records = unchanged_records.select(
    'GenotypeName', 'StemAnthocyanin', 'AnthocyaninIntensity', 
    'StemPubescence', 'StemPubescenceIntensity', 'LeafBladeColour', 
    'LeafBladeColourIntensity', 'ColourOfVein', 'IntensityOfColour8', 
    'SpineonLeaf', 'FlowerColour', 'FruitingPattern', 
    'FruitColour', 'IntensityofColour13', 'Stripes', 
    'StripesDensity', 'FruitPatches', 'SpinesInCalyx', 
    'DensityOfSpininess', 'FruitShape', 'surrogate_key', 
    'effective_date', 'end_date', 'active_flag', 'hash_md5','Action'
).drop('hash_md5')
display(final_unchanged_data_records)

GenotypeName,StemAnthocyanin,AnthocyaninIntensity,StemPubescence,StemPubescenceIntensity,LeafBladeColour,LeafBladeColourIntensity,ColourOfVein,IntensityOfColour8,SpineonLeaf,FlowerColour,FruitingPattern,FruitColour,IntensityofColour13,Stripes,StripesDensity,FruitPatches,SpinesInCalyx,DensityOfSpininess,FruitShape,surrogate_key,effective_date,end_date,active_flag,Action
102,Present,Medium,Present,Medium,Green,Medium,Purple,Medium,Absent,Purple,Solitary,Purple,Dark,Present,Sparse,Absent,Absent,Absent,Globular,2,2024-08-13,9999-12-31,true,NOCHANGE
104,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Greenish_White,Solitary,Green,Med ium,Absent,Absent,Absent,Present,Absent,Club,4,2024-08-13,9999-12-31,true,NOCHANGE
105,Present,Weak,Present,Medium,Green,Medium,Purple,Light,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate,5,2024-08-13,9999-12-31,true,NOCHANGE
106,Present,Medium,Present,Weak,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Absent,Absent,Absent,Present,Medium,Club,6,2024-08-13,9999-12-31,true,NOCHANGE
109,Present,Medium,Present,Medium,Green,Dark,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate,9,2024-08-13,9999-12-31,true,NOCHANGE
110,Absent,Absent,Present,Strong,Green,Medium,Green,Medium,Absent,Purple,Solitary,Green,Medium,Absent,Absent,Present,Absent,Absent,Globular,10,2024-08-13,9999-12-31,true,NOCHANGE


In [0]:
Inserted_records = column_renamer(merged_df.filter(col("action") == 'INSERT'), suffix="_source", append=False).select('GenotypeName', 'StemAnthocyanin', 'AnthocyaninIntensity', 'StemPubescence', 'StemPubescenceIntensity', 'LeafBladeColour', 'LeafBladeColourIntensity', 'ColourOfVein', 'IntensityOfColour8', 'SpineonLeaf', 'FlowerColour', 'FruitingPattern', 'FruitColour', 'IntensityofColour13', 'Stripes', 'StripesDensity', 'FruitPatches', 'SpinesInCalyx', 'DensityOfSpininess', 'FruitShape','Action')
display(Inserted_records)

GenotypeName,StemAnthocyanin,AnthocyaninIntensity,StemPubescence,StemPubescenceIntensity,LeafBladeColour,LeafBladeColourIntensity,ColourOfVein,IntensityOfColour8,SpineonLeaf,FlowerColour,FruitingPattern,FruitColour,IntensityofColour13,Stripes,StripesDensity,FruitPatches,SpinesInCalyx,DensityOfSpininess,FruitShape,Action
111,Present,Weak,Present,Medium,Green,Medium,Green,Medium,Absent,Purple,Solitary,Purple,Medium,Present,Strong,Absent,Present,Medium,Ovoid,INSERT
112,Absent,Absent,Present,Medium,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Present,Sparse,Present,Absent,Absent,Obovate,INSERT


In [0]:
max_sk = active_target_rename_df.agg({"surrogate_key_target": "max"}).collect()[0][0]
print(max_sk)

10


In [0]:
from pyspark.sql.functions import row_number, col, lit, date_format, current_date
from pyspark.sql.window import Window

window_spec = Window.orderBy("GenotypeName")

insert_records = Inserted_records.withColumn("row_number", row_number().over(window_spec))\
 .withColumn("surrogate_key", col("row_number") + max_sk)\
 .withColumn("effective_date", date_format(current_date(), DATE_FORMAT))\
 .withColumn("end_date", date_format(lit(future_date), DATE_FORMAT))\
 .withColumn("active_flag", lit(True))\
 .drop("row_number")

display(insert_records)

GenotypeName,StemAnthocyanin,AnthocyaninIntensity,StemPubescence,StemPubescenceIntensity,LeafBladeColour,LeafBladeColourIntensity,ColourOfVein,IntensityOfColour8,SpineonLeaf,FlowerColour,FruitingPattern,FruitColour,IntensityofColour13,Stripes,StripesDensity,FruitPatches,SpinesInCalyx,DensityOfSpininess,FruitShape,Action,surrogate_key,effective_date,end_date,active_flag
111,Present,Weak,Present,Medium,Green,Medium,Green,Medium,Absent,Purple,Solitary,Purple,Medium,Present,Strong,Absent,Present,Medium,Ovoid,INSERT,11,2024-08-13,9999-12-31,true
112,Absent,Absent,Present,Medium,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Present,Sparse,Present,Absent,Absent,Obovate,INSERT,12,2024-08-13,9999-12-31,true


In [0]:
delete_records = column_renamer(merged_df.filter(col("action") == 'DELETE'), suffix="_target", append=False).select('GenotypeName', 'StemAnthocyanin', 'AnthocyaninIntensity', 'StemPubescence', 'StemPubescenceIntensity', 'LeafBladeColour', 'LeafBladeColourIntensity', 'ColourOfVein', 'IntensityOfColour8', 'SpineonLeaf', 'FlowerColour', 'FruitingPattern', 'FruitColour', 'IntensityofColour13', 'Stripes', 'StripesDensity', 'FruitPatches', 'SpinesInCalyx', 'DensityOfSpininess', 'FruitShape','surrogate_key','effective_date','Action').withColumn("end_date", date_format(current_date(),DATE_FORMAT)).withColumn("active_flag", lit(False))
display(delete_records)

GenotypeName,StemAnthocyanin,AnthocyaninIntensity,StemPubescence,StemPubescenceIntensity,LeafBladeColour,LeafBladeColourIntensity,ColourOfVein,IntensityOfColour8,SpineonLeaf,FlowerColour,FruitingPattern,FruitColour,IntensityofColour13,Stripes,StripesDensity,FruitPatches,SpinesInCalyx,DensityOfSpininess,FruitShape,surrogate_key,effective_date,Action,end_date,active_flag
103,Absent,Absent,Present,Medium,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Present,Sparse,Present,Absent,Absent,Obovate,3,2024-08-13,DELETE,2024-08-13,false
107,Present,Weak,Present,Medium,Green,Medium,Green,Medium,Absent,Purple,Solitary,Purple,Medium,Present,Strong,Absent,Present,Medium,Ovoid,7,2024-08-13,DELETE,2024-08-13,false


In [0]:
print(merged_df.columns)

['GenotypeName_target', 'StemAnthocyanin_target', 'AnthocyaninIntensity_target', 'StemPubescence_target', 'StemPubescenceIntensity_target', 'LeafBladeColour_target', 'LeafBladeColourIntensity_target', 'ColourOfVein_target', 'IntensityOfColour8_target', 'SpineonLeaf_target', 'FlowerColour_target', 'FruitingPattern_target', 'FruitColour_target', 'IntensityofColour13_target', 'Stripes_target', 'StripesDensity_target', 'FruitPatches_target', 'SpinesInCalyx_target', 'DensityOfSpininess_target', 'FruitShape_target', 'surrogate_key_target', 'effective_date_target', 'end_date_target', 'active_flag_target', 'hash_md5_target', 'GenotypeName_source', 'StemAnthocyanin_source', 'AnthocyaninIntensity_source', 'StemPubescence_source', 'StemPubescenceIntensity_source', 'LeafBladeColour_source', 'LeafBladeColourIntensity_source', 'ColourOfVein_source', 'IntensityOfColour8_source', 'SpineonLeaf_source', 'FlowerColour_source', 'FruitingPattern_source', 'FruitColour_source', 'IntensityofColour13_source', 

In [0]:
update_records = column_renamer(merged_df.filter(col("action") == 'UPDATE'), suffix="_target", append=False).select('GenotypeName', 'StemAnthocyanin', 'AnthocyaninIntensity', 'StemPubescence', 'StemPubescenceIntensity', 'LeafBladeColour', 'LeafBladeColourIntensity', 'ColourOfVein', 'IntensityOfColour8', 'SpineonLeaf', 'FlowerColour', 'FruitingPattern', 'FruitColour', 'IntensityofColour13', 'Stripes', 'StripesDensity', 'FruitPatches', 'SpinesInCalyx', 'DensityOfSpininess', 'FruitShape','surrogate_key', 'effective_date','Action') \
    .withColumn("end_date", date_format(current_date(),DATE_FORMAT)).withColumn("active_flag", lit(False)) \
    .unionByName(
            column_renamer(merged_df.filter(col("action") == 'UPDATE'), suffix="_source", append=False)\
                .select('GenotypeName', 'StemAnthocyanin', 'AnthocyaninIntensity', 'StemPubescence', 'StemPubescenceIntensity', 'LeafBladeColour', 'LeafBladeColourIntensity', 'ColourOfVein', 'IntensityOfColour8', 'SpineonLeaf', 'FlowerColour', 'FruitingPattern', 'FruitColour', 'IntensityofColour13', 'Stripes', 'StripesDensity', 'FruitPatches', 'SpinesInCalyx', 'DensityOfSpininess', 'FruitShape','Action')\
                .withColumn("effective_date",date_format(current_date(),DATE_FORMAT))\
                .withColumn("end_date",date_format(lit(future_date),DATE_FORMAT))\
                .withColumn("row_number",row_number().over(window_spec))\
                .withColumn("surrogate_key",col("row_number")+ max_sk)\
                .withColumn("active_flag", lit(True))\
                .drop("row_number")
                )
display(update_records)

GenotypeName,StemAnthocyanin,AnthocyaninIntensity,StemPubescence,StemPubescenceIntensity,LeafBladeColour,LeafBladeColourIntensity,ColourOfVein,IntensityOfColour8,SpineonLeaf,FlowerColour,FruitingPattern,FruitColour,IntensityofColour13,Stripes,StripesDensity,FruitPatches,SpinesInCalyx,DensityOfSpininess,FruitShape,surrogate_key,effective_date,Action,end_date,active_flag
101,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Purple,Cluster,Purple,Medium,Present,Sparse,Present,Absent,Absent,Obovate,1,2024-08-13,UPDATE,2024-08-13,false
108,Present,Strong,Present,Medium,Green,Light,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Present,Present,Medium,Ovoid,8,2024-08-13,UPDATE,2024-08-13,false
101,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Purple,Cluster,Purple,Medium,Present,Sparse,Present,Absent,Absent,Globular,11,2024-08-13,UPDATE,9999-12-31,true
108,Present,Strong,Present,Medium,Green,Light,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Present,Present,Medium,Club,12,2024-08-13,UPDATE,9999-12-31,true


In [0]:
Finalised_df = final_unchanged_data_records\
    .unionByName(insert_records)\
    .unionByName(update_records)\
    .unionByName(delete_records)

display(Finalised_df)

GenotypeName,StemAnthocyanin,AnthocyaninIntensity,StemPubescence,StemPubescenceIntensity,LeafBladeColour,LeafBladeColourIntensity,ColourOfVein,IntensityOfColour8,SpineonLeaf,FlowerColour,FruitingPattern,FruitColour,IntensityofColour13,Stripes,StripesDensity,FruitPatches,SpinesInCalyx,DensityOfSpininess,FruitShape,surrogate_key,effective_date,end_date,active_flag,Action
102,Present,Medium,Present,Medium,Green,Medium,Purple,Medium,Absent,Purple,Solitary,Purple,Dark,Present,Sparse,Absent,Absent,Absent,Globular,2,2024-08-13,9999-12-31,true,NOCHANGE
104,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Greenish_White,Solitary,Green,Med ium,Absent,Absent,Absent,Present,Absent,Club,4,2024-08-13,9999-12-31,true,NOCHANGE
105,Present,Weak,Present,Medium,Green,Medium,Purple,Light,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate,5,2024-08-13,9999-12-31,true,NOCHANGE
106,Present,Medium,Present,Weak,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Absent,Absent,Absent,Present,Medium,Club,6,2024-08-13,9999-12-31,true,NOCHANGE
109,Present,Medium,Present,Medium,Green,Dark,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Obovate,9,2024-08-13,9999-12-31,true,NOCHANGE
110,Absent,Absent,Present,Strong,Green,Medium,Green,Medium,Absent,Purple,Solitary,Green,Medium,Absent,Absent,Present,Absent,Absent,Globular,10,2024-08-13,9999-12-31,true,NOCHANGE
111,Present,Weak,Present,Medium,Green,Medium,Green,Medium,Absent,Purple,Solitary,Purple,Medium,Present,Strong,Absent,Present,Medium,Ovoid,11,2024-08-13,9999-12-31,true,INSERT
112,Absent,Absent,Present,Medium,Green,Medium,Green,Medium,Absent,Light_Purple,Cluster,Green,Medium,Present,Sparse,Present,Absent,Absent,Obovate,12,2024-08-13,9999-12-31,true,INSERT
101,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Purple,Cluster,Purple,Medium,Present,Sparse,Present,Absent,Absent,Obovate,1,2024-08-13,2024-08-13,false,UPDATE
108,Present,Strong,Present,Medium,Green,Light,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Present,Present,Medium,Ovoid,8,2024-08-13,2024-08-13,false,UPDATE
